### Overview

In [1]:
import pandas as pd
import numpy as np

### Data Wrangling

In this research, the three most essential variables we are going to use are QS ranking (Y), GDP per capita (X) and manufacturing export (IV) of countries. In this section, we will be doing data wrangling to datasets to aquire the variables of interest.

#### step1: head the dataset to have an overview of it

In [2]:
ranking = pd.read_csv('qs-world-university-rankings-2017-to-2022-V2.csv')

In [3]:
ranking.head()

,university,year,rank_display,score,link,country,city,region,logo,type,research_output,student_faculty_ratio,international_students,size,faculty_count
0,Massachusetts Institute of Technology (MIT),2017,1,100.0,https://www.topuniversities.com/universities/m...,United States,Cambridge,North America,https://www.topuniversities.com/sites/default/...,Private,Very High,4.0,"3,730",M,"3,065"
1,Stanford University,2017,2,98.7,https://www.topuniversities.com/universities/s...,United States,Stanford,North America,https://www.topuniversities.com/sites/default/...,Private,Very High,3.0,"3,879",L,"4,725"
2,Harvard University,2017,3,98.3,https://www.topuniversities.com/universities/h...,United States,Cambridge,North America,https://www.topuniversities.com/sites/default/...,Private,Very High,5.0,"5,877",L,"4,646"
3,University of Cambridge,2017,4,97.2,https://www.topuniversities.com/universities/u...,United Kingdom,Cambridge,Europe,https://www.topuniversities.com/sites/default/...,Public,Very high,4.0,"7,925",L,"5,800"
4,California Institute of Technology (Caltech),2017,5,96.9,https://www.topuniversities.com/universities/c...,United States,Pasadena,North America,https://www.topuniversities.com/sites/default/...,Private,Very High,2.0,692,S,968


In [4]:
ranking.dtypes

university                 object
year                        int64
rank_display               object
score                     float64
link                       object
country                    object
city                       object
region                     object
logo                       object
type                       object
research_output            object
student_faculty_ratio     float64
international_students     object
size                       object
faculty_count              object
dtype: object

#### step2: extract top 200 ranking universities of each year and add ranking index to the universities in each year, then aggregate them by country

In [5]:
# create a dictionary to store the top 200 universities for each year
top_200_per_year = {}
# create a list to store the aggregated data frames for all years
aggregated_dataframes_all_years = []
for year_number in range(2017, 2023):  # Include 2022 as well
    # Using boolean indexing to select rows for each year
    data_year=ranking[ranking['year'] == year_number] 
    # Select columns of interest
    data_year = data_year[['university', 'year', 'rank_display', 'country', 'region']]
    # Convert rank_display to a numeric value
    data_year['rank_display'] = pd.to_numeric(data_year['rank_display'], errors='coerce')
    # Sort the data by the numeric rank and keep the top 200 universities
    top_200 = data_year.sort_values('rank_display').head(200)
    # Generate the ranking index in reverse to make sure higher ranking has bigger index
    top_200['ranking index'] = np.arange(200, 0, -1)[:len(top_200)]
    # Store in the top_200_per_year dictionary
    top_200_per_year[year_number] = top_200
    # Sort the ranking index by country and sum them up
    aggregated_data = top_200.groupby('country')['ranking index'].sum().reset_index()
    # Add a year column to the aggregated data
    aggregated_data['year'] = year_number
    # Rearrange columns in the order: country, year, ranking index
    aggregated_data = aggregated_data[['country', 'year', 'ranking index']]
    # Append to the list
    aggregated_dataframes_all_years.append(aggregated_data)

#### step3: combine the data and select countries that appear in all years

In [6]:
# Concatenate all the data frames in the list into a single data frame
combined_aggregated_data = pd.concat(aggregated_dataframes_all_years, ignore_index=True)
# Keep the countries that appear in all six years
countries_in_all_years = combined_aggregated_data['country'].value_counts()
countries_in_all_years = countries_in_all_years[countries_in_all_years == 6].index.tolist()
# Filter the combined_aggregated_data to include only these countries
filtered_aggregated_data = combined_aggregated_data[combined_aggregated_data['country'].isin(countries_in_all_years)]
# The filtered_aggregated_data now contains only the countries that appear in all 6 years (from 2017 to 2022)

In [8]:
filtered_aggregated_data.head()

,country,year,ranking index
0,Argentina,2017,116
1,Australia,2017,1110
2,Austria,2017,63
3,Belgium,2017,257
4,Brazil,2017,90


Add covariates from another 2 datasets

In [9]:
variable_data = pd.read_csv('variable_data_copy.csv')

In [10]:
variable_data.head()

,Country,Code,ContinentCode,Year,Economic growth: the rate of change of real GDP,Gross Domestic Product billions of 2010 U.S. dollars,Unemployment rate,Exports of goods and services billion USD,Exports of goods and services annual growth,Current account balance billion USD
0,Argentina,ARG,SA,2017,2.82,598.8,8.35,72.86,2.62,-31.15
1,Argentina,ARG,SA,2018,-2.62,583.1,9.22,75.77,0.65,-27.08
2,Argentina,ARG,SA,2019,-2.00,571.5,9.84,80.26,9.75,-3.49
3,Argentina,ARG,SA,2020,-9.94,514.6,11.46,64.04,-17.71,3.12
4,Argentina,ARG,SA,2021,10.40,568.1,8.74,87.87,9.22,6.71


In [11]:
new_variable = pd.read_csv('new_variable_copy.csv')
new_variable.head()

,Country,Code,ContinentCode,Year,GDP per capita constant 2010 dollars,Capital investment as percent of GDP,Capital investment billion USD,Household consumption as percent of GDP,Household consumption billion USD,Labor force million people,Government spending as percent of GDP,Government spending billion USD,Population growth percent,Happiness Index 0 (unhappy) - 10 (happy)
0,Argentina,ARG,SA,2017,13595.04,18.21,117.22,66.74,429.55,19.58,17.70,113.90,1.04,6.60
1,Argentina,ARG,SA,2018,13105.40,16.61,87.19,69.47,364.59,20.10,15.81,82.95,1.02,6.39
2,Argentina,ARG,SA,2019,12716.22,14.21,63.63,66.13,296.09,20.61,16.44,73.63,0.99,6.09
3,Argentina,ARG,SA,2020,11341.27,14.13,54.48,63.79,245.94,19.41,16.89,65.12,0.97,5.93
4,Argentina,ARG,SA,2021,12402.49,17.47,85.14,60.89,296.69,21.19,15.84,77.19,0.95,5.97


In [25]:
#make sure the column names align with the ones in filtered_aggregated_data
variable_data.rename(columns={'Country': 'country'}, inplace=True)
variable_data.rename(columns={'Year': 'year'}, inplace=True)
new_variable.rename(columns={'Country': 'country'}, inplace=True)
new_variable.rename(columns={'Year': 'year'}, inplace=True)

In [14]:
variable_data['country'] = variable_data['country'].replace({
    'China': 'China (Mainland)',
    'Hong Kong': 'Hong Kong SAR',
    'USA': 'United States'
})

In [15]:
new_variable['country'] = new_variable['country'].replace({
    'China': 'China (Mainland)',
    'Hong Kong': 'Hong Kong SAR',
    'USA': 'United States'
})

In [26]:
# Merge the two dataframes
merged_data = pd.merge(variable_data, filtered_aggregated_data, on=['country', 'year'], how='left')
merged_data = pd.merge(new_variable, merged_data, on=['country', 'year',"Code","ContinentCode"], how='left')

### merged_data dataframe will now contain all columns of variable_data and new_variable, plus 'ranking index' from filtered_aggregated_data for each country and each year

In [27]:
merged_data.head()

,country,Code,ContinentCode,year,GDP per capita constant 2010 dollars,Capital investment as percent of GDP,Capital investment billion USD,Household consumption as percent of GDP,Household consumption billion USD,Labor force million people,...,Government spending billion USD,Population growth percent,Happiness Index 0 (unhappy) - 10 (happy),Economic growth: the rate of change of real GDP,Gross Domestic Product billions of 2010 U.S. dollars,Unemployment rate,Exports of goods and services billion USD,Exports of goods and services annual growth,Current account balance billion USD,ranking index
0,Argentina,ARG,SA,2017,13595.04,18.21,117.22,66.74,429.55,19.58,...,113.90,1.04,6.60,2.82,598.8,8.35,72.86,2.62,-31.15,116
1,Argentina,ARG,SA,2018,13105.40,16.61,87.19,69.47,364.59,20.10,...,82.95,1.02,6.39,-2.62,583.1,9.22,75.77,0.65,-27.08,126
2,Argentina,ARG,SA,2019,12716.22,14.21,63.63,66.13,296.09,20.61,...,73.63,0.99,6.09,-2.00,571.5,9.84,80.26,9.75,-3.49,128
3,Argentina,ARG,SA,2020,11341.27,14.13,54.48,63.79,245.94,19.41,...,65.12,0.97,5.93,-9.94,514.6,11.46,64.04,-17.71,3.12,128
4,Argentina,ARG,SA,2021,12402.49,17.47,85.14,60.89,296.69,21.19,...,77.19,0.95,5.97,10.40,568.1,8.74,87.87,9.22,6.71,133


In [28]:
merged_data.to_csv('merged_data.csv', index=False)

### Modeling

In [2]:
!pip install linearmodels

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from linearmodels.iv import IV2SLS
from sklearn import (
    linear_model, metrics, neural_network, pipeline, model_selection
)
import seaborn as sns

In [4]:
df = pd.read_csv('merged_data.csv')
print(df.columns)

Index(['country', 'Code', 'ContinentCode', 'year',
       'GDP per capita constant 2010 dollars',
       'Capital investment as percent of GDP',
       'Capital investment billion USD',
       'Household consumption as percent of GDP',
       'Household consumption billion USD', 'Labor force million people',
       'Government spending as percent of GDP',
       'Government spending billion USD', 'Population growth percent',
       'Happiness Index 0 (unhappy) - 10 (happy)',
       'Economic growth: the rate of change of real GDP',
       'Gross Domestic Product billions of 2010 U.S. dollars',
       'Unemployment rate', 'Exports of goods and services billion USD',
       'Exports of goods and services annual growth',
       'Current account balance billion USD', 'ranking index'],
      dtype='object')


#### Part 1: Linear regression using variables selected by Lasso

$ log (Ranking Index) = \beta_0 + \beta_1 (\text{GDP}) + \beta_2 (\text{GDP growth}) + \beta_3 x_3 + \cdots + \beta_n x_n + \varepsilon $

In [5]:
# as the 'Capital investment billion USD' and 'Capital investment as percent of GDP' represents similar things 
# and have a relatively high correlation, we kept one of them 
X = df[['Current account balance billion USD',
        'Exports of goods and services annual growth',
        'Exports of goods and services billion USD',
        'Unemployment rate',
        'Gross Domestic Product billions of 2010 U.S. dollars',
        'Economic growth: the rate of change of real GDP',
        'Happiness Index 0 (unhappy) - 10 (happy)',
        'Population growth percent',
        'Government spending as percent of GDP',
        'Labor force million people',
        'Household consumption as percent of GDP',
        'Capital investment billion USD',
        'GDP per capita constant 2010 dollars']]

y = df['ranking index']
y_log = np.log(y + 1)

df['const'] = 1
reg1 = sm.OLS(endog = y_log, exog = X, \
    missing='drop')
results = reg1.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:          ranking index   R-squared (uncentered):                   0.979
Model:                            OLS   Adj. R-squared (uncentered):              0.978
Method:                 Least Squares   F-statistic:                              648.1
Date:                Tue, 28 Nov 2023   Prob (F-statistic):                   6.07e-143
Time:                        01:06:03   Log-Likelihood:                         -235.09
No. Observations:                 192   AIC:                                      496.2
Df Residuals:                     179   BIC:                                      538.5
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                                                           coef    std err          t      P>|t|      [0.025      0.975]

In [6]:
# calculating mse 
sqft_lr_model = linear_model.LinearRegression()
sqft_lr_model.fit(X, y_log)
mse = metrics.mean_squared_error(y_log, sqft_lr_model.predict(X))
print(mse)

0.6429409949562513


- As we could see from the OLS result table above, the coefficient on the variable 'Gross Domestic Product billions of 2010 U.S. dollars' is -8.911e-05, the associated p-value is 0.304, which is statistically insignificant at the 5% level. 
- And the coefficient on the variable 'Economic growth: the rate of change of real GDP' is -0.0014, the associated p-value is 0.959, which is statistically insignificant at the 5% level as well.
- But the coefficient on 'GDP per capita constant 2010 dollars' has a p-value of 0, which is statistically significant at the 5% level.

In [7]:
# fitted graph

#### Part 2: Two-stage least squares (2SLS) regression

- As the OLS model above is likely suffer from endogeneity issues (reverse causality: better university is likely lead to a higher GDP of the country, and omitted variable bias : there are too many variables correlated with both the university ranking of a country and its GDP, and we may not be able to get data and control all of them). Therefore, here we include the 2SLS model to deal with the problem of endogeneity.

- In this model, to test the effect of 'Gross Domestic Product billions of 2010 U.S. dollars' on log of ranking index, the instrument we have chosen is 'Exports of goods and services billion USD', as it satisfies the three conditions for instruments, which are first stage, exogeneity and exclusion.

##### 1. first stage

$$
\text{Gross Domestic Product billions of 2010 U.S. dollars}_i = \delta_0 + \delta_1 \text{Exports of goods and services billion USD}_i + v_i
$$

In [8]:
# test the first stage
results_fs = sm.OLS(df['Gross Domestic Product billions of 2010 U.S. dollars'],
                    df[['const', 'Exports of goods and services billion USD']]).fit()
print(results_fs.summary())

                                             OLS Regression Results                                             
Dep. Variable:     Gross Domestic Product billions of 2010 U.S. dollars   R-squared:                       0.749
Model:                                                              OLS   Adj. R-squared:                  0.747
Method:                                                   Least Squares   F-statistic:                     566.3
Date:                                                  Tue, 28 Nov 2023   Prob (F-statistic):           6.80e-59
Time:                                                          01:06:20   Log-Likelihood:                -1735.8
No. Observations:                                                   192   AIC:                             3476.
Df Residuals:                                                       190   BIC:                             3482.
Df Model:                                                             1                         

- As we see from the table, the coefficient is large and the p-value is 0 which is lower than 0.05, therefore the instrument is correlated with the GDP. -> satisfies the first condition for instrument we mentioned above

- We cannot directly test whether the instrument is correlated with the error term or not (exogeneity and exclusion). But intuitively, the export should not correlated with the ranking index except for the fact that it inflences GDP. As we could see from the QS ranking calculation, none of the considered factors (Sustainability, Employment outcomes, International research network, etc.) seem related to the export of countries. Therefore we could infer export is a viable instrument in this case.

##### 2. second stage

$$
\log(\text{ranking index})_i = \beta_0 + \beta_1 \widehat{\text{GDP}}_i + u_i
$$

In [9]:
df['predicted_gdp'] = results_fs.predict()

results_ss = sm.OLS(y_log,
                    df[['const', 'predicted_gdp']]).fit()
print(results_ss.summary())

                            OLS Regression Results                            
Dep. Variable:          ranking index   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     97.27
Date:                Tue, 28 Nov 2023   Prob (F-statistic):           8.65e-19
Time:                        01:06:27   Log-Likelihood:                -290.06
No. Observations:                 192   AIC:                             584.1
Df Residuals:                     190   BIC:                             590.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             5.0544      0.094     53.847

##### 3. directly using linearmodels package (IV2SLS)

In [10]:
iv = IV2SLS(dependent = y_log,
            exog = df['const'],
            endog = df['Gross Domestic Product billions of 2010 U.S. dollars'],
            instruments = df['Exports of goods and services billion USD']).fit(cov_type='unadjusted')

print(iv.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:          ranking index   R-squared:                      0.2503
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2463
No. Observations:                 192   F-statistic:                    86.714
Date:                Tue, Nov 28 2023   P-value (F-stat)                0.0000
Time:                        01:06:34   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                                                  Parameter Estimates                                                   
                                                      Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------------------------------------------
const

- the package give us the same coefficient as we get from the first stage second stage analysis, therefore in the next part we directly use the iv package

##### 4. testing the effect of GDP growth on ranking index

- for testing the impact of GDP gowth on log ranking index, we change the instrument to 'Exports of goods and services annual growth'

In [11]:
# first stage
results_fs = sm.OLS(df['Economic growth: the rate of change of real GDP'],
                    df[['const', 'Exports of goods and services annual growth']]).fit()
print(results_fs.summary())

                                           OLS Regression Results                                          
Dep. Variable:     Economic growth: the rate of change of real GDP   R-squared:                       0.607
Model:                                                         OLS   Adj. R-squared:                  0.605
Method:                                              Least Squares   F-statistic:                     293.3
Date:                                             Tue, 28 Nov 2023   Prob (F-statistic):           2.25e-40
Time:                                                     01:06:43   Log-Likelihood:                -448.93
No. Observations:                                              192   AIC:                             901.9
Df Residuals:                                                  190   BIC:                             908.4
Df Model:                                                        1                                         
Covariance Type:            

In [ ]:
# 2SLS
iv2 = IV2SLS(dependent = y_log,
            exog = df['const'],
            endog = df['Economic growth: the rate of change of real GDP'],
            instruments = df['Exports of goods and services annual growth']).fit(cov_type='unadjusted')

print(iv2.summary)